In [1]:
import h5py

import itertools as it
import numpy as np

from IPython.display import display, HTML

In [2]:
dt = np.int
data = [[1,2,3],[1,2,3,4,5]]
data = np.array([np.array([i for i in a],dtype=dt) for a in data])

with h5py.File('myfile.hdf5','w') as f:
    h5dt = h5py.special_dtype(vlen=dt)
    dset = f.create_dataset('vlen_int', (5,), dtype=h5dt)
    dset[:] = np.random.rand(5,5)*10

In [3]:
with h5py.File('myfile.hdf5','r') as f:
    df = f['vlen_int']
    display(df[:])

array([array([1, 7, 3, 9, 8]), array([6, 2, 2, 8, 4]),
       array([4, 9, 7, 8, 5]), array([1, 6, 3, 5, 9]),
       array([6, 6, 7, 9, 5])], dtype=object)

In [4]:
arr = [[1],[1,2]]

with h5py.File('myfile.hdf5','w') as f:
    dt1 = h5py.special_dtype(vlen=h5py.special_dtype(enum=('i', dict(foo=1, bar=2))))
    df1 = f.create_dataset('test', (2,), dtype=dt1)
    df1[:] = np.array(arr)

with h5py.File('myfile.hdf5','r') as f:
    df2 = f['test']
    assert([e.tolist() for e in df2[:]] == arr)
    assert(h5py.check_dtype(enum=h5py.check_dtype(vlen=dt1)) ==
                     h5py.check_dtype(enum=h5py.check_dtype(vlen=df2.dtype)))




In [5]:
names = ["foo","bar","baz"]
nmap = dict((n,i) for i,n in enumerate(names))

dt = h5py.special_dtype(enum=(np.int32, nmap))

pset = list(it.chain(([a] for a in names),
                     (list(a) for a in it.permutations(names,2))))

with h5py.File('myfile.hdf5','w') as f:
    dtl = h5py.special_dtype(vlen=dt)
    df = f.create_dataset('pset', (len(pset),), dtype=dtl)
    for i,p in enumerate(pset):
        df[i] = [nmap[n] for n in p]

display(h5py.check_dtype(enum=h5py.check_dtype(vlen=dtl)))

{'bar': 1, 'baz': 2, 'foo': 0}

In [6]:
with h5py.File('myfile.hdf5','r') as f:
    df = f['pset']
    display(df[:])

display(h5py.check_dtype(enum=h5py.check_dtype(vlen=df.dtype)))

array([array([0], dtype=int32), array([1], dtype=int32),
       array([2], dtype=int32), array([0, 1], dtype=int32),
       array([0, 2], dtype=int32), array([1, 0], dtype=int32),
       array([1, 2], dtype=int32), array([2, 0], dtype=int32),
       array([2, 1], dtype=int32)], dtype=object)

{'bar': 1, 'baz': 2, 'foo': 0}